In [1]:
import pandas as pd
import os
from langdetect import detect
import numpy as np
from collections import Counter

In [2]:
parent_folder = "./DevGPT"

folder_names = ["snapshot_20230803", "snapshot_20230824", "snapshot_20230914",
                "snapshot_20230810", "snapshot_20230831", "snapshot_20231012",
                "snapshot_20230727", "snapshot_20230817", "snapshot_20230907"]

# Define the patterns to identify different types of files
patterns = ["_commit_sharings.json", "_file_sharings.json"]

# Initialize dataframes for each type of file
commit_sharings_df = pd.DataFrame()
file_sharings_df = pd.DataFrame()

# Iterate through folders and files
for folder_name in folder_names:
    full_folder_path = os.path.join(parent_folder, folder_name)
    for file_name in os.listdir(full_folder_path):
        file_path = os.path.join(full_folder_path, file_name)
        
        # Check if the file name contains any of the specified patterns
        for pattern in patterns:
            if pattern in file_name:
                # Create custom location string for reading CSV files
                custom_location = f"{file_path}"
                print(custom_location)
                # Read CSV file into a dataframe
                df = pd.read_json(custom_location)
                
                # Assign the dataframe to the corresponding variable based on the pattern
                if "_commit_sharings.json" in pattern:
                    commit_sharings_df = pd.concat([commit_sharings_df, df], ignore_index=True)
                elif "_file_sharings.json" in pattern:
                    file_sharings_df = pd.concat([file_sharings_df, df], ignore_index=True)


./DevGPT/snapshot_20230803/20230803_095317_commit_sharings.json
./DevGPT/snapshot_20230803/20230803_103605_file_sharings.json
./DevGPT/snapshot_20230824/20230824_102435_commit_sharings.json
./DevGPT/snapshot_20230824/20230824_111114_file_sharings.json
./DevGPT/snapshot_20230914/20230914_104122_file_sharings.json
./DevGPT/snapshot_20230914/20230914_083202_commit_sharings.json
./DevGPT/snapshot_20230810/20230810_124807_commit_sharings.json
./DevGPT/snapshot_20230810/20230810_133121_file_sharings.json
./DevGPT/snapshot_20230831/20230831_063412_commit_sharings.json
./DevGPT/snapshot_20230831/20230831_072722_file_sharings.json
./DevGPT/snapshot_20231012/20231012_230826_commit_sharings.json
./DevGPT/snapshot_20231012/20231012_234250_file_sharings.json
./DevGPT/snapshot_20230727/20230727_200102_file_sharings.json
./DevGPT/snapshot_20230727/20230727_200003_commit_sharings.json
./DevGPT/snapshot_20230817/20230817_151344_file_sharings.json
./DevGPT/snapshot_20230817/20230817_131244_commit_sharin

In [3]:
def extract_json_to_columns(df):
    df = pd.DataFrame(df.Sources.tolist())
    df = pd.concat([df, pd.DataFrame(df.ChatgptSharing.str[0].tolist()).add_prefix("CGS_")], axis=1)
    return df

In [4]:
commit_sharings_df = extract_json_to_columns(commit_sharings_df)
file_sharings_df = extract_json_to_columns(file_sharings_df)

### Remove Null values from Conversations

In [5]:
commit_sharings_df['CGS_Conversations'] = commit_sharings_df.CGS_Conversations.fillna('').apply(list)
file_sharings_df['CGS_Conversations'] = file_sharings_df.CGS_Conversations.fillna('').apply(list)

### Extract Prompts and Answers from Conversations

In [6]:
# Function to extract Prompts and Answers
def extract_prompts_and_answers(conversations):
    prompts = []
    answers = []

    if conversations:
        for i, conv in enumerate(conversations):
                prompt = conv.get("Prompt", "")
                answer = conv.get("Answer", "")
                prompts.append(prompt)
                answers.append(answer)
    
    conv_len = len(prompts) + len(answers)
    return pd.Series({"Prompts": prompts, "Answers": answers, "Conversation_Length": conv_len})

##### Apply on all dataframes

In [8]:
commit_sharings_df[["Prompts", "Answers", "Conversation_Length"]] = commit_sharings_df.CGS_Conversations.apply(extract_prompts_and_answers)
file_sharings_df[["Prompts", "Answers", "Conversation_Length"]] = file_sharings_df.CGS_Conversations.apply(extract_prompts_and_answers)

### Duplicate Records Cleaning

In [9]:
def dedupe_data(data, identifier='CGS_URL'):
    return data.loc[data.groupby(identifier).Conversation_Length.idxmax()]

In [10]:
commit_sharings_final_df = dedupe_data(commit_sharings_df)
file_sharings_final_df = dedupe_data(file_sharings_df)

In [11]:
print(f"commit_sharings - Original: {commit_sharings_df.shape}, Final: {commit_sharings_final_df.shape}")
print(f"file_sharings - Original: {file_sharings_df.shape}, Final: {file_sharings_final_df.shape}")

commit_sharings - Original: (3245, 25), Final: (672, 25)
file_sharings - Original: (8664, 29), Final: (1668, 29)


### Make final Dataframe

In [12]:
columns_to_keep = [
    'Type', 'RepoLanguage', 'CGS_URL', 'CGS_Status', 'CGS_DateOfConversation', 'CGS_DateOfAccess', 
    'CGS_Title', 'CGS_NumberOfPrompts', 'CGS_TokensOfPrompts', 
    'CGS_TokensOfAnswers', 'CGS_Model', 
    'Prompts', 'Answers', 'Conversation_Length', 'CGS_Conversations'
]

In [13]:
final_df = pd.concat(
    [
     commit_sharings_final_df[columns_to_keep], 
     file_sharings_final_df[columns_to_keep]
    ], 
    axis=0,
    ignore_index=True,
    
)

In [14]:
final_df = final_df.drop_duplicates(subset="CGS_URL", keep="first")

In [15]:
final_df.drop(final_df[final_df.Conversation_Length == 0].index, inplace=True)

In [16]:
def detect_language(text):
    text = ' '.join(text)
    try:
        return detect(text)
    except:
        return 'Error'

In [17]:
final_df['language'] = final_df['Prompts'].apply(detect_language)

In [18]:
final_df = final_df[final_df.language == 'en']

In [19]:
final_df.head()

,Type,RepoLanguage,CGS_URL,CGS_Status,CGS_DateOfConversation,CGS_DateOfAccess,CGS_Title,CGS_NumberOfPrompts,CGS_TokensOfPrompts,CGS_TokensOfAnswers,CGS_Model,Prompts,Answers,Conversation_Length,CGS_Conversations,language
0,commit,CSS,https://chat.openai.com/share/0009359a-f33c-4d...,200,"August 8, 2023",2023-08-24 10:23:21.867744,Replace Getting Started section.,2.0,841.0,880.0,Default,[# Working set\n\ndocs/README.md:\n```\nWarn: ...,"[[CODE_BLOCK_0], Certainly! Here's the updated...",4,[{'Prompt': '# Working set docs/README.md: ``...,en
1,commit,CSS,https://chat.openai.com/share/002ec31f-83bd-44...,200,"August 4, 2023",2023-09-14 08:22:00.979886,Implement Clipboard Fallback.,1.0,449.0,420.0,Default,[# Working set\n\nsrc/frontend/components/Exec...,[[CODE_BLOCK_0]],2,[{'Prompt': '# Working set src/frontend/compo...,en
2,commit,CSS,https://chat.openai.com/share/006570dd-ce57-48...,200,"August 21, 2023",2023-08-24 10:21:31.568574,Refine Gitignore Behavior,1.0,731.0,262.0,Default,"[You are Junior, an AI system aiding developer...",[[CODE_BLOCK_0]],2,"[{'Prompt': 'You are Junior, an AI system aidi...",en
3,commit,CSS,https://chat.openai.com/share/008ea3c2-4cf5-4f...,200,"July 21, 2023",2023-08-03 09:49:45.851175,Create execute endpoint.,2.0,653.0,808.0,,[# Working set\n\nsrc/execute/executeAndForwar...,"[[CODE_BLOCK_0], I apologize for the confusion...",4,[{'Prompt': '# Working set src/execute/execut...,en
4,commit,CSS,https://chat.openai.com/share/00b111a5-00b5-46...,200,"August 4, 2023",2023-08-24 10:23:56.168934,Update div styling App.jsx.,1.0,516.0,358.0,Default,[# Working set\n\nsrc/frontend/App.jsx:\n```\n...,[[CODE_BLOCK_0]],2,[{'Prompt': '# Working set src/frontend/App.j...,en


### Kaggle Dataset

In [20]:
import json
# Define the file path
file_path = "./ChatGPT-89k/chatlogs.jsonl"  # Replace with the path to your JSONL file

# Initialize an empty list to store the data
data = []

# Read the JSONL file line by line
with open(file_path, 'r') as file:
    for line in file:
        # Parse the JSON object from each line
        json_data = json.loads(line)
        data.append(json_data)

In [21]:
kaggle_df = pd.DataFrame(data)

In [22]:
kaggle_df['post_number'] = kaggle_df['post_number'].apply(lambda x: x[0])

In [23]:
# Function to extract Prompts and Answers
def extract_prompts_and_answers_kaggle(conversations):
    prompts = []
    answers = []

    if conversations:
        for conv in conversations:
            if conv['user'][0] == 'Anonymous':
                prompts.append(conv['message'])
            elif conv['user'][0] == 'Chat GPT':
                answers.append(conv['message'])
    
    conv_len = len(prompts) + len(answers)
    return pd.Series({"Prompts": prompts, "Answers": answers, "Conversation_Length": conv_len})

In [24]:
kaggle_df[["Prompts", "Answers", "Conversation_Length"]] = kaggle_df.conversation.apply(extract_prompts_and_answers_kaggle)

In [25]:
software_engineering_keywords = [
    "software engineering",
    "software development",
    "programming",
    "coding",
    "computer science",
    "algorithms",
    "data structures",
    "version control",
    "agile methodology",
    "scrum",
    "kanban",
    "continuous integration",
    "continuous deployment",
    "DevOps",
    "unit testing",
    "integration testing",
    "system testing",
    "debugging",
    "refactoring",
    "code review",
    "software architecture",
    "design patterns",
    "object-oriented programming",
    "functional programming",
    "web development",
    "mobile development",
    "cloud computing",
    "containerization",
    "microservices",
    "API design",
    "database management",
    "security",
    "networking",
    "deployment",
    "maintenance",
    "documentation",
    "software lifecycle",
    "requirements engineering",
    "user experience design",
    "user interface design",
    "software project management",
    "software quality assurance",
    "software testing",
    "software metrics",
    "software tools",
    "open source",
    "proprietary software",
    "software patents",
    "software licensing",
    "software industry",
    "software trends",
    "emerging technologies",
    "artificial intelligence",
    "machine learning",
    "data science",
    "blockchain",
    "internet of things",
    "cybersecurity",
    # Programming Languages
    "Python",
    "Java",
    "JavaScript",
    "C++",
    "C#",
    "PHP",
    "Ruby",
    "Swift",
    "Kotlin",
    "SQL",
    "TypeScript",
    "Scala",
    "Perl",
    "Rust",
    "Haskell",
    "Objective-C",
    "Dart",
    "Lua",
    "Assembly",
    "VB.NET",
    "MATLAB",
    "Shell scripting",
    "HTML",
    "CSS"
]

In [26]:
def contains_keyword(text, keywords):
    for keyword in keywords:
        if keyword.lower() in text:
            return True
    return False

In [27]:
filtered_df = kaggle_df[kaggle_df['Prompts'].apply(lambda x: contains_keyword(' '.join(x).lower().split(' '), software_engineering_keywords)) & 
                 kaggle_df['Answers'].apply(lambda x: contains_keyword(' '.join(x).lower().split(' '), software_engineering_keywords))]

In [28]:
filtered_df = filtered_df[(filtered_df.Conversation_Length % 2 == 0) & (filtered_df.Conversation_Length <= 16) & (filtered_df.Conversation_Length >=2)]

In [29]:
filtered_df['language'] = filtered_df['Prompts'].apply(detect_language)

In [30]:
filtered_df = filtered_df[filtered_df.language == 'en']

In [31]:
final_kaggle_df = filtered_df.sample(n=2200)

### Merge Kaggle and DevGPT Data

In [32]:
final_df.columns

Index(['Type', 'RepoLanguage', 'CGS_URL', 'CGS_Status',
       'CGS_DateOfConversation', 'CGS_DateOfAccess', 'CGS_Title',
       'CGS_NumberOfPrompts', 'CGS_TokensOfPrompts', 'CGS_TokensOfAnswers',
       'CGS_Model', 'Prompts', 'Answers', 'Conversation_Length',
       'CGS_Conversations', 'language'],
      dtype='object')

In [33]:
final_kaggle_df.columns

Index(['post_number', 'conversation', 'Prompts', 'Answers',
       'Conversation_Length', 'language'],
      dtype='object')

In [34]:
merged_dataset = pd.concat([final_df[['Prompts', 'Answers', 'Conversation_Length']], final_kaggle_df[['Prompts', 'Answers', 'Conversation_Length']]], axis=0)

In [35]:
final_dataset = merged_dataset.reset_index().drop(columns=["index"]).reset_index().rename(columns={"index": "chat_id"}).sample(n=1000)

### RQ1

In [36]:
rq1 = final_dataset.copy()

In [37]:
rq1['First_Prompt'] = [prompt[0] if prompt else '' for prompt in final_dataset.Prompts]

In [38]:
rq1[["chat_id", "First_Prompt"]].to_csv("RQ1.csv", index=False)

### RQ2

In [39]:
rq2 = final_dataset.copy()

In [40]:
rq2['NoOfTurns'] = rq2.apply(lambda x: (len(x.Prompts) + len(x.Answers))/2, axis=1)

In [41]:
rq2 = rq2[(rq2.NoOfTurns >= 2) & (rq2.NoOfTurns <= 8)]

In [42]:
rq2 = rq2.explode('Prompts')
rq2['prompt_no'] = rq2.groupby('chat_id').cumcount() + 1

In [43]:
rq2[["chat_id", "Prompts", "prompt_no"]].to_csv('RQ2.csv', index=False)